In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')

current_date=Timestamp('2023-06-24 04:00:00')


In [4]:
import os 
os.chdir('/Users/akhileshvarmabhupathiraju/Documents/taxi_ml_project/NYC')


In [7]:

from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/49352
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-05-27 04:00:00 to 2023-06-24 03:00:00


2023-06-24 01:14:49,433 INFO: USE `ny_taxi_wala_featurestore`
2023-06-24 01:14:49,799 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `ny_taxi_wala_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-05-26 04:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-06-25 03:00:00.000'


In [11]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/49352
Connected. Call `.close()` to terminate connection gracefully.

In [10]:
model

NameError: name 'model' is not defined

In [12]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2023-06-24 04:00:00
1,2,0.0,2023-06-24 04:00:00
2,3,0.0,2023-06-24 04:00:00
3,4,6.0,2023-06-24 04:00:00
4,5,0.0,2023-06-24 04:00:00
...,...,...,...
256,261,3.0,2023-06-24 04:00:00
257,262,4.0,2023-06-24 04:00:00
258,263,20.0,2023-06-24 04:00:00
259,264,8.0,2023-06-24 04:00:00


In [13]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/49352
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/49352/fs/48208/fg/66014


Uploading Dataframe: 0.00% |          | Rows 0/261 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/49352/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x287734880>, None)